In [67]:
import pandas as pd
import requests
import sqlite3
from sqlalchemy import create_engine
import json
import os
import time
import pandas as pd
from sqlalchemy import engine as sql
from IPython import display

def getPg(page = 0): 
    # Справочник для параметров GET-запроса
    params = {
        'text': 'middle Python developer', # В имени должно быть слово "middle Python developer"
        'search_field':'name',
        'area': 113,
        'page': page, # Индекс страницы поиска
        'per_page': 20 # Кол-во вакансий на 1 странице
    }
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # чтобы Кириллица отображалась корректно
    req.close()
    return data
# Считываем вакансии
for page in range(0, 5):
    # Преобразуем текст ответа запроса в справочник Python
    jsObjct = json.loads(getPg(page))
    # Сохраняем файл в папку
    FileName = './docs/pages/{}.json'.format(len(os.listdir('./docs/pages')))
    # Создаем новый документ, записываем в него
    f = open(FileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObjct, ensure_ascii=False))
    f.close()
    # Проверка на последнюю страницу, если вакансий меньше 2000
    if (jsObjct['pages'] - page) <= 1:
        break
    
    # задержка
    time.sleep(1)
print('Страница поиска загружена')
for fl in os.listdir('./docs/pages'):
    # Открываем файл, читаем его содержимое, закрываем файл
    f = open('./docs/pages/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
    # Преобразуем полученный текст в объект справочника
    jsonObj = json.loads(jsonText)
    # Получаем и проходимся по списку вакансий
    for v in jsonObj['items']:
        #получаем детальную информацию o вакансии
        req = requests.get(v['url'])
        data = req.content.decode()
        req.close()
        # Создаем файл в формате json с id вакансии в качестве названия, пишем в него и закрываем файл
        fileName = './docs/vacancies/{}.json'.format(v['id'])
        f = open(fileName, mode='w', encoding='utf8')
        f.write(data)
        f.close()
        time.sleep(1)
    
print('Вакансии собраны')

conn = sqlite3.connect('hw1.db')
cursor = conn.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS vacanciesAPI (company_name WARCHAR(200), position WARCHAR(200), job_description WARCHAR(2000), key_skills WARCHAR(200))')
conn.commit()
sql_query = 'INSERT INTO vacanciesAPI (company_name, position, job_description, key_skills) VALUES (?, ?, ?, ?)'

# Создаем списки для столбцов таблицы vacanciesAPI
company_names = [] # Список названий компаний
positions = [] # Список наименований вакансий
descriptions = [] # Список описаний вакансий
key_skills = [] # Список названий навыков

# Счетчик обработанных файлов
docs = len(os.listdir('./docs/vacancies'))
i = 0

# Проходимся по всем файлам в папке vacancies
for fl in os.listdir('./docs/vacancies'):
    
    # Открываем, читаем и закрываем файл
    with open('./docs/vacancies/{}'.format(fl), encoding='utf8') as f:
        jsonText = f.read()
        f.close()
    
    # Текст файла переводим в справочник
    jsonvac = json.loads(jsonText)
    
    # Заполняем списки для таблиц
    company_names.append(jsonvac['employer']['name'])
    positions.append(jsonvac['name'])
    descriptions.append(jsonvac['description'])
    key_skills.append(', '.join([i['name'] for i in jsonvac['key_skills']]))
    
    # Увеличиваем счетчик обработанных файлов на 1, очищаем вывод ячейки и выводим прогресс
    i += 1
    display.clear_output(wait=True)
    display.display('Готово {} из {}'.format(i, docs))

# Создаем пандосовский датафрейм, который затем сохраняем в БД в таблицу vacancies
df = pd.DataFrame({'company_name': company_names, 'position': positions, 'job_description': descriptions, 'key_skills': key_skills})
df.to_sql('vacanciesAPI', conn, if_exists='append', index=False)

# Закрываем соединение с БД
conn.close()

# Выводим сообщение об окончании программы
display.clear_output(wait=True)
display.display('Вакансии загружены в БД')
engine = create_engine('sqlite:///hw1.db')
df = pd.read_sql_query('SELECT * FROM vacanciesAPI', engine)
print("Количество вакансий в таблице:", len(df))
engine.dispose()

'Вакансии загружены в БД'

Количество вакансий в таблице: 100
